In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import pickle

In [3]:
import matplotlib.pyplot as plt

In [4]:
import json
import string
from string import punctuation

In [5]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

In [6]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')


import sys
import os

def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    l = []
    for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
        l.append(subtree.leaves())
    
    return l

def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    #word = stemmer.stem(word)
    #word = lemmatizer.lemmatize(word)
    return word

def acceptable_word(word):
    """Checks conditions for acceptable word: length, stopword."""
    accepted = bool(2 <= len(word) <= 40
        and word.lower() not in stopwords)
    return accepted


def get_terms(tree):
    kp = []
    for leaf in leaves(tree):
        term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
        if term:
            kp.append(term)
    
    return kp


In [7]:
def get_kp(text):
    
    # Used when tokenizing words
    sentence_re = r'''(?x)      # set flag to allow verbose regexps
            (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
          | \w+(?:-\w+)*        # words with optional internal hyphens
          | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
          | \.\.\.              # ellipsis
          | [][.,;"'?():_`-#]    # these are separate tokens; includes ], [
        '''

    #lemmatizer = nltk.WordNetLemmatizer()
    #stemmer = nltk.stem.porter.PorterStemmer()

    #Taken from Su Nam Kim Paper
    grammar = r"""
        NBAR:
            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
            {<JJ|JJR|JJS|VBG|VBN>*<NN|NNS|NNP|NNPS|VBG>}
        NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """

    #toks = nltk.regexp_tokenize(text, sentence_re)
    postoks = nltk.tag.pos_tag(text)
    chunker = nltk.RegexpParser(grammar)
    tree = chunker.parse(postoks)
    terms = get_terms(tree)
    #pos,pos_set = find_positions(text,terms)
    return terms#,pos,pos_set

In [8]:
data = 'Inspec/docsutf8/'
keys = 'Inspec/keys/'
files = os.listdir(data)
key_files = os.listdir(keys)

In [9]:
def find_positions(document,bert_tocs, kps):
    ''' 
    merge the same kps & keep present kps in document
    Inputs:
        document : a word list : ['sun', 'sunshine', ...] || lower cased
        kps : can have more than one kp : [['sun'], ['key','phrase'], ['sunshine']] || not duplicate
    Outputs:
        all_present_kps : present keyphrases
        positions_for_all : start_end_posisiton for prensent keyphrases
        a present kp postions list : every present's positions in documents, 
        each kp can be presented in several postions .
        [[[0,0],[20,21]], [[1,1]]]
    '''
    tot_doc_char = ' '.join(document)
    
    positions_for_all = []
    position_start,position_end =[],[]
    all_present_kps = []
    for kp in kps:
        ans_string = ' '.join(kp)
        
        if ans_string not in tot_doc_char:
            continue
        else: 
            positions_for_each = []
            # find all positions for each kp
            for i in range(0, len(bert_tocs) - len(kp) + 1):
                found = False
                search_str = ''
                if ans_string.startswith(bert_tocs[i]):
                    found = True
                    search_str +=bert_tocs[i]
                    search_idx = i
                    while found and search_idx<(len(bert_tocs)-1):
                        search_idx+=1
                        if search_str+bert_tocs[search_idx] in ans_string:
                            search_str+=bert_tocs[search_idx]
                        elif search_str+' '+bert_tocs[search_idx] in ans_string:
                            search_str+=' '+bert_tocs[search_idx]
                        else:
                            found = False
                        
                if (search_str==ans_string) and (i<search_idx):
                    assert len(kp) >= 1
                    positions_for_each.append((i+1, search_idx))
                    position_start.append(i+1)
                    position_end.append(search_idx)
                    
        if len(positions_for_each) > 0 :
            positions_for_all.extend(positions_for_each)
            all_present_kps.append(kp)
           
    assert len(positions_for_all) >= len(all_present_kps)
    
    if len(all_present_kps) == 0:
        return [None,None]
    return [position_start,position_end],set(positions_for_all)

In [10]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

def remove_stopwords(text):
    text=[word for word in text if word not in stopword]
    return text

In [11]:
def calculate_f1(y_labels,y_preds,depth,levels):
    precision = []
    recall = []
    f1 = []
    
    for idx,y_label in enumerate(y_labels):
        tp = 0
        p = []
        r = []
        y_label = set(np.where(y_label==1)[0])
        
        #print(y_preds[idx].shape)
        key_idx = np.argsort(-y_preds[idx])#[:,0])
        #print(sorted(-y_preds[idx]))
        y_new = np.sort(y_preds[idx])[::-1]
        #print(y_preds[idx])
        preds = key_idx[y_new>=0.5]
        for i in range(depth):
            if len(preds)>i:
                if preds[i] in y_label:
                    tp+=1
            p.append(tp/(min(i,len(preds))+1))
            r.append(tp/max(len(y_label),1))
    
    
        level_index = []
        level_p = []
        level_r = []
        for idx,level in enumerate(levels):
            level_p.append(p[level-1])
            level_r.append(r[level-1])
            if p[level-1]+r[level-1]>0:
                level_index.append(2*p[level-1]*r[level-1]/(p[level-1]+r[level-1]))
            else:
                level_index.append(0)
        #print('k',level_index)
        precision.append(level_p)
        recall.append(level_r)
        f1.append(level_index)
    precision = np.array(precision)
    recall = np.array(recall)
    f1 = np.array(f1)
    
    print('F1',np.mean(f1,axis=0),np.mean(precision,axis=0),np.mean(recall,axis=0))
    
                
            

In [12]:
vocab = "D:/Word embedding/bert/assets/vocab.txt"
tokenizer = BertWordPieceTokenizer(vocab, lowercase=True)
encoder = TFBertModel.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [13]:
x_train = tf.convert_to_tensor(np.loadtxt('Data/KP20/x_train.csv'))
x_mask = tf.convert_to_tensor(np.loadtxt('Data/KP20/x_mask_train.csv'))
with open('Data/KP20/x_pos_train.pkl','rb') as f:
    x_pos = tf.convert_to_tensor(pickle.load(f))
    print(x_pos.shape)

with open('Data/KP20/y_train.pkl','rb') as f:
    y_train = tf.convert_to_tensor(pickle.load(f))
    print(y_train.shape)

(15487, 2, 258)
(15487, 258, 2)


In [14]:
max_len = x_train.shape[-1]
max_kp = y_train.shape[1]

In [15]:
input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
embedding = encoder(input_ids, attention_mask=attention_mask)[0]

bilstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(36,
                                                             kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.00,stddev=0.15),
                                                             dropout = 0.15,
                                                             return_sequences=True),
                                                             merge_mode=None)(embedding)
pos_mask = tf.keras.layers.Input(shape=(2,max_kp),dtype='int32')
mask_start = pos_mask[0][0]
mask_end = pos_mask[0][1]

start_rep_fr = tf.gather(bilstm1[0],mask_start,axis=1)
start_rep_bk = tf.gather(bilstm1[1],mask_start,axis=1)
end_rep_fr = tf.gather(bilstm1[0],mask_end,axis=1)
end_rep_bk = tf.gather(bilstm1[0],mask_end,axis=1)


span_fe_diff_fr = start_rep_fr-end_rep_fr
span_fe_prod_fr = tf.math.multiply(start_rep_fr,end_rep_fr)
span_fe_diff_bk = start_rep_bk-end_rep_bk
span_fe_prod_bk = tf.math.multiply(start_rep_bk,end_rep_bk)


span_fe = tf.keras.layers.concatenate([start_rep_fr,
                     end_rep_fr,
                     start_rep_bk,
                     end_rep_bk,
                     span_fe_diff_fr,
                     span_fe_diff_bk,
                     span_fe_prod_fr,
                     span_fe_prod_bk
                    ],2)

bilstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(6,return_sequences=True,dropout = 0.25,
                                                            #kernel_initializer=tf.keras.initializers.(mean=0.0,stddev=0.05),
                                                            ),
                                        
                                         merge_mode='concat',
                                         input_shape=(max_kp,30*4))(span_fe)
output = tf.keras.layers.Dense(1,activation='sigmoid')(bilstm2)

In [16]:
kpe_model = tf.keras.models.Model(inputs=[input_ids,attention_mask,pos_mask], outputs=output)
kpe_model.layers[3].trainable = False

In [17]:
print(kpe_model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 2, 258)]     0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 512, 768), ( 109482240   input_1[0][0]                    
                                                                 input_2[0][0]         

In [18]:
tf.reduce_mean(y_train[:,:,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.16838203>

In [19]:
import gc
gc.collect()

40

In [20]:
loss_object = tf.keras.losses.CategoricalCrossentropy()


def loss_function(y_true, y_pred):
    #print(y_pred.shape,y_true.shape)
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    y_true = tf.clip_by_value(y_true-1, 0, 1)
    #loss_ = loss_object(y_true[mask], y_pred[mask])
    pos = tf.math.logical_and(tf.math.equal(y_true, 1),mask)
    loss_ = -tf.reduce_sum(20.0*tf.math.log(y_pred[pos]+0.01))-tf.reduce_sum(tf.math.log(0.99-y_pred[tf.logical_not(pos)]))
    #mask = tf.cast(mask, dtype=loss_.dtype)
    #loss_ *= mask
    #loss_ = tf.boolean_mask(loss_,mask)
    
    return loss_#tf.reduce_mean(tf.reduce_sum(loss_,axis=1)/tf.reduce_sum(mask,axis=1))


def ac_metrics(y_true,y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    y_true = tf.cast(tf.clip_by_value(y_true-1, 0, 1),dtype='int32')
    y_pred = tf.cast(tf.where(y_pred>=0.5,1,0),dtype='int32')
    diff = tf.cast(tf.math.equal(y_true,y_pred),dtype='int32')
    mask = tf.cast(mask, dtype=diff.dtype)
    diff*= mask
    
    return tf.reduce_mean(tf.reduce_sum(diff,axis=1)/tf.reduce_sum(mask,axis=1))

In [21]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00009)
kpe_model.compile(optimizer=opt,
              loss=loss_function,
              metrics=[ac_metrics])

#checkpoint
filepath="weights-improvement-{epoch:02d}-{val_ac_metrics:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_ac_metrics', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [80]:
history = kpe_model.fit([x_train[1000:],x_mask[1000:],x_pos[1000:]], y_train[1000:,:,0], 
                          batch_size=36,epochs=1,#callbacks=callbacks_list, 
                          #use_multiprocessing=True, 
                         validation_split=0.05)


164/383 [===========>..................] - ETA: 3s - loss: 4254.2114 - ac_metrics: 0.892 - ETA: 12:03 - loss: 3639.7097 - ac_metrics: 0.71 - ETA: 16:01 - loss: 3300.6658 - ac_metrics: 0.59 - ETA: 17:59 - loss: 3143.0601 - ac_metrics: 0.62 - ETA: 19:06 - loss: 3145.4731 - ac_metrics: 0.61 - ETA: 19:52 - loss: 3040.2185 - ac_metrics: 0.61 - ETA: 20:22 - loss: 3047.3337 - ac_metrics: 0.59 - ETA: 20:45 - loss: 3101.5347 - ac_metrics: 0.56 - ETA: 21:01 - loss: 3121.8105 - ac_metrics: 0.56 - ETA: 21:14 - loss: 3164.9258 - ac_metrics: 0.54 - ETA: 21:23 - loss: 3143.9009 - ac_metrics: 0.57 - ETA: 21:30 - loss: 3133.8403 - ac_metrics: 0.56 - ETA: 21:36 - loss: 3128.4995 - ac_metrics: 0.58 - ETA: 21:40 - loss: 3105.3926 - ac_metrics: 0.57 - ETA: 21:43 - loss: 3144.5977 - ac_metrics: 0.55 - ETA: 21:46 - loss: 3166.0168 - ac_metrics: 0.54 - ETA: 21:48 - loss: 3182.7170 - ac_metrics: 0.54 - ETA: 21:49 - loss: 3132.1121 - ac_metrics: 0.53 - ETA: 21:49 - loss: 3107.2310 - ac_metrics: 0.54 - ETA: 21:5

328/383 [========================>.....] - ETA: 13:46 - loss: 3156.5535 - ac_metrics: 0.63 - ETA: 13:43 - loss: 3154.0234 - ac_metrics: 0.63 - ETA: 13:39 - loss: 3157.5559 - ac_metrics: 0.63 - ETA: 13:35 - loss: 3156.3374 - ac_metrics: 0.63 - ETA: 13:31 - loss: 3156.0483 - ac_metrics: 0.63 - ETA: 13:28 - loss: 3158.2039 - ac_metrics: 0.63 - ETA: 13:24 - loss: 3160.4314 - ac_metrics: 0.63 - ETA: 13:20 - loss: 3158.5474 - ac_metrics: 0.63 - ETA: 13:17 - loss: 3159.3379 - ac_metrics: 0.64 - ETA: 13:13 - loss: 3166.2126 - ac_metrics: 0.64 - ETA: 13:09 - loss: 3167.4543 - ac_metrics: 0.64 - ETA: 13:05 - loss: 3165.7036 - ac_metrics: 0.64 - ETA: 13:02 - loss: 3172.8943 - ac_metrics: 0.64 - ETA: 12:58 - loss: 3171.2112 - ac_metrics: 0.64 - ETA: 12:54 - loss: 3166.5056 - ac_metrics: 0.64 - ETA: 12:51 - loss: 3165.7107 - ac_metrics: 0.64 - ETA: 12:47 - loss: 3165.6504 - ac_metrics: 0.64 - ETA: 12:43 - loss: 3163.3623 - ac_metrics: 0.64 - ETA: 12:39 - loss: 3165.3132 - ac_metrics: 0.64 - ETA: 12

383/383 [==============================] - ETA: 3:26 - loss: 3086.2478 - ac_metrics: 0.647 - ETA: 3:22 - loss: 3087.9927 - ac_metrics: 0.647 - ETA: 3:19 - loss: 3088.2471 - ac_metrics: 0.647 - ETA: 3:15 - loss: 3087.4541 - ac_metrics: 0.646 - ETA: 3:11 - loss: 3085.6831 - ac_metrics: 0.646 - ETA: 3:07 - loss: 3086.1162 - ac_metrics: 0.645 - ETA: 3:03 - loss: 3086.2603 - ac_metrics: 0.645 - ETA: 2:59 - loss: 3085.9402 - ac_metrics: 0.645 - ETA: 2:56 - loss: 3084.1743 - ac_metrics: 0.646 - ETA: 2:52 - loss: 3084.2305 - ac_metrics: 0.646 - ETA: 2:48 - loss: 3083.4424 - ac_metrics: 0.646 - ETA: 2:44 - loss: 3082.9905 - ac_metrics: 0.645 - ETA: 2:40 - loss: 3083.1130 - ac_metrics: 0.645 - ETA: 2:37 - loss: 3082.0823 - ac_metrics: 0.645 - ETA: 2:33 - loss: 3081.7405 - ac_metrics: 0.645 - ETA: 2:29 - loss: 3081.5730 - ac_metrics: 0.645 - ETA: 2:25 - loss: 3079.8193 - ac_metrics: 0.645 - ETA: 2:21 - loss: 3079.8838 - ac_metrics: 0.645 - ETA: 2:17 - loss: 3079.5564 - ac_metrics: 0.645 - ETA: 2:

In [ ]:
plt.plot(history.history['ac_metrics'])
plt.plot(history.history['val_ac_metrics'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [81]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for test")
predictions = kpe_model.predict([x_train[:500],x_mask[:500],x_pos[:500]])
print("predictions shape:", predictions.shape)


Generate predictions for test
predictions shape: (500, 258, 1)


In [82]:
calculate_f1(np.where(y_train[:500,:,0]-1==1,1,0),predictions[:,:,0],25,[5,10,15])

F1 [0.05293348 0.06307821 0.06892498] [0.05416667 0.0511627  0.05116007] [0.07254705 0.13008821 0.17917682]


In [83]:
kpe_model.save('./checkpoints/KP20/kp20_final')


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./checkpoints/KP20/kp20_final\assets


In [84]:
t = 198
np.where(predictions[t,:,0][np.where((y_train[t,:,0])>0)]>=0.5)

(array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 22, 23, 24, 25, 26, 28], dtype=int64),)

In [85]:
np.argsort(-predictions[t,:,0])

array([  3,  18,   4,   2,   1,  25,  24,  20,  11,  14,  23,  28,   7,
        17,  13,  10,   9,  15,  22,   0,  26,   8,   5,  19,  12,  16,
        21,  29,  27,   6,  32,  33,  37,  34,  53,  54,  52,  55,  56,
        51,  57,  50,  58,  49,  59,  60,  48,  61,  47,  62,  31,  46,
        63,  30,  64,  45,  65,  44,  66,  35,  36,  43,  67,  68,  42,
        69,  41,  70,  40,  71,  39,  38,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 257, 113, 114, 256,
       115, 116, 255, 117, 118, 254, 119, 120, 253, 121, 122, 252, 123,
       124, 251, 125, 126, 250, 127, 128, 249, 129, 130, 248, 131, 132,
       247, 133, 246, 134, 135, 245, 136, 137, 244, 138, 139, 243, 140,
       141, 242, 142, 143, 241, 144, 145, 240, 146, 147, 148, 239, 149,
       150, 151, 238, 152, 153, 154, 155, 237, 156, 157, 158, 15

In [86]:
np.where((y_train[t,:,0]-1)==1)

(array([18], dtype=int64),)

In [87]:
import collections

In [143]:
#t = 123
idx = np.argsort(-predictions[t,:,0])
preds = np.where(predictions[t,np.argsort(-predictions[t,:,0])]>=0.5)
scores = predictions [t,idx]
res = np.array(x_pos[t])[:,idx[preds[0]]]
text = tokenizer.decode(list(tf.cast(x_train[t,:],dtype='int32')))
print(text)
keyphrases = collections.defaultdict(float)
for i in range(res.shape[1]):
    phrase = res[:,i]
    phrase = tokenizer.decode(list(tf.cast(x_train[t,phrase[0]:(phrase[1]+1)],dtype='int32')))
    keyphrases[phrase] = max(keyphrases[phrase],scores[i][0])

keyphrases

parametric models for sequential data such as hidden markov models stochastic contextfree grammars and linear dynamical systems are widely used in timeseries analysis and structural data analysis computation of the likelihood function is one of primary considerations in many learning methods iterative calculation of the likelihood such as the model selection is still timeconsuming though there are effective algorithms based on dynamic programming the present paper studies parameter learning in a simplified feature space to reduce the computational cost simplifying data is a common technique seen in feature selection and dimension reduction though an oversimplified space causes adverse learning results therefore we mathematically investigate a condition of the feature map to have an asymptotically equivalent convergence point of estimated parameters referred to as the vicarious map as a demonstration to find vicarious maps we consider the feature space which limits the length of data an

defaultdict(float,
            {'learning': 0.5971708,
             'feature selection': 0.580973,
             'feature space': 0.5754552,
             'timeseries analysis': 0.56864053,
             'dimension reduction': 0.5616109,
             'dynamic programming': 0.5573249,
             'length': 0.55653286,
             'vicarious map': 0.5516192,
             'condition': 0.5503541,
             'equivalent convergence point': 0.54687923,
             'demonstration': 0.5431143,
             'linear dynamical systems': 0.5389345,
             'likelihood': 0.53707206,
             'adverse learning': 0.5360124,
             'sequential data': 0.5341229,
             'timeconsuming': 0.5260348,
             'common technique': 0.5251576,
             'simplified feature space': 0.5241617,
             'data': 0.5188727,
             'many learning': 0.516537,
             'parameters': 0.5082375,
             'vicarious maps': 0.50619435})